In [1]:
import pandas as pd

data0 = pd.read_json(r"C:\Users\maazk\Videos\C, C++ and HTML\Project\data-processing\data\Final data\AAPL.json")
df = pd.DataFrame(data0)
df.head()

FileNotFoundError: File C:\Users\maazk\Videos\C, C++ and HTML\Project\data-processing\data\Final data\AAPL.json does not exist

In [4]:
import yfinance as yf 
import datetime

start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2024, 11, 4)

data = yf.download('AAPL', start=start, end=end)
df2 = pd.DataFrame(data)
df2


C:\Users\sheet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-02,187.149994,188.440002,183.889999,185.639999,184.938217,82488700
2024-01-03,184.220001,185.880005,183.429993,184.250000,183.553482,58414500
2024-01-04,182.149994,183.089996,180.880005,181.910004,181.222321,71983600
2024-01-05,181.990005,182.759995,180.169998,181.179993,180.495087,62303300
2024-01-08,182.089996,185.600006,181.500000,185.559998,184.858521,59144500
...,...,...,...,...,...,...
2024-10-28,233.320007,234.729996,232.550003,233.399994,233.399994,36087100
2024-10-29,233.100006,234.330002,232.320007,233.669998,233.669998,35417200
2024-10-30,232.610001,233.470001,229.550003,230.100006,230.100006,47070900


In [5]:
df2.drop(['Close', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
df2

,Open
Date,
2024-01-02,187.149994
2024-01-03,184.220001
2024-01-04,182.149994
2024-01-05,181.990005
2024-01-08,182.089996
...,...
2024-10-28,233.320007
2024-10-29,233.100006
2024-10-30,232.610001


In [6]:
def Add_Trend_Column(df):
    # Shift Close prices to compare with previous day's Close
    df['Prev_Open'] = df['Open'].shift(1)
    
    # Compute difference between today's and previous day's Close price
    df['Diff'] = df['Open'] - df['Prev_Open']
    
    # Determine Trend based on difference
    df['Trend'] = df['Diff'].apply(lambda x: 'rise' if x > 0 else 'fall' if x < 0 else 'equal')
    
    # Drop intermediate columns
    df = df.drop(columns=['Prev_Open', 'Diff'])
    
    return df
    
df2 = Add_Trend_Column(df2)
df2.head()

,Open,Trend
Date,,
2024-01-02,187.149994,equal
2024-01-03,184.220001,fall
2024-01-04,182.149994,fall
2024-01-05,181.990005,fall
2024-01-08,182.089996,rise


In [7]:
df2.reset_index(inplace=True)
# Rename the 'Date' column in df2 to match the 'date' column in df1
df2.rename(columns={'Date': 'date'}, inplace=True)



In [8]:
# Merge the two DataFrames on the 'date' column
merged_df = pd.merge(df, df2[['date', 'Open','Trend']], on='date', how='left')

# Display the merged DataFrame
merged_df

NameError: name 'df' is not defined

In [ ]:
none_sentiment_rows = merged_df[merged_df['news'].apply(lambda x: any(news.get('sentiment') is None for news in x))]
none_sentiment_rows

,date,ticker,news,Open,Trend
29,2024-01-30,AAPL,"[{'index': 550, 'time': '00:00:00', 'title': '...",190.940002,fall


In [ ]:
# Assuming merged_df is already defined

def calculate_average_sentiment(news_list):
    total_sentiment = {'polarity': 0, 'neg': 0, 'neu': 0, 'pos': 0}
    count = 0

    for news in news_list:
        if news is not None:
            sentiment = news.get('sentiment')  # Use get to avoid KeyError
            if sentiment:  # Check if sentiment is not None and not empty 
                total_sentiment['polarity'] += sentiment.get('polarity', 0)
                total_sentiment['neg'] += sentiment.get('neg', 0)
                total_sentiment['neu'] += sentiment.get('neu', 0)
                total_sentiment['pos'] += sentiment.get('pos', 0)
                count += 1  # Increment count only if valid sentiment exists

    # Calculate average
    if count > 0:
        average_sentiment = {k: v / count for k, v in total_sentiment.items()}
    else:
        average_sentiment = {k: 0 for k in total_sentiment}  # Default to zero if no valid entries

    return average_sentiment

# Calculate sentiment for each row in merged_df
merged_df['average_sentiment'] = merged_df['news'].apply(calculate_average_sentiment)

In [ ]:
merged_df.columns

Index(['date', 'ticker', 'news', 'Open', 'Trend', 'sentiment'], dtype='object')

In [ ]:
merged_df.drop(['sentiment'], axis=1, inplace=True)
merged_df

,date,ticker,news,Open,Trend,average_sentiment
0,2024-01-01,AAPL,"[{'index': 0, 'time': '06:21:00', 'title': '2 ...",NaN,NaN,"{'polarity': 0.992625, 'neg': 0.043375, 'neu':..."
1,2024-01-02,AAPL,"[{'index': 8, 'time': '00:00:00', 'title': 'AA...",187.149994,equal,"{'polarity': 0.6988644067796609, 'neg': 0.0305..."
2,2024-01-03,AAPL,"[{'index': 67, 'time': '00:00:00', 'title': ' ...",184.220001,fall,"{'polarity': 0.7423684210526316, 'neg': 0.0278..."
3,2024-01-04,AAPL,"[{'index': 86, 'time': '00:00:00', 'title': ' ...",182.149994,fall,"{'polarity': 0.6239166666666667, 'neg': 0.0482..."
4,2024-01-05,AAPL,"[{'index': 98, 'time': '00:00:00', 'title': '3...",181.990005,fall,"{'polarity': 0.6653125, 'neg': 0.029, 'neu': 0..."
...,...,...,...,...,...,...
246,2024-09-03,AAPL,"[{'index': 3401, 'time': '00:00:00', 'title': ...",228.550003,fall,"{'polarity': 0.45575000000000004, 'neg': 0.079..."
247,2024-09-04,AAPL,"[{'index': 3409, 'time': '00:00:00', 'title': ...",221.660004,fall,"{'polarity': 0.573857142857143, 'neg': 0.02828..."
248,2024-09-05,AAPL,"[{'index': 3416, 'time': '00:00:00', 'title': ...",221.630005,fall,"{'polarity': 0.9093333333333333, 'neg': 0.0352..."
249,2024-09-06,AAPL,"[{'index': 3437, 'time': '00:00:00', 'title': ...",223.949997,rise,"{'polarity': 0.9117142857142857, 'neg': 0.0227..."


In [ ]:
print(merged_df['news'][0])

[{'index': 0, 'time': '06:21:00', 'title': '2 FAANG Stocks to Buy in 2024 and 1 to Avoid', 'content': 'The stock market has soared over the last decade. A $10,000 investment made in a market-tracking fund like the Vanguard S&P 500 ETF(NYSEMKT: VOO) at the start of 2014 would be worth $29,673 today after having grown at a compound annual rate of 11.6%, assuming that dividends were reinvested along the way.\n\nBut the handful of elite stocks known as the FAANG group has delivered even stronger returns. In the last decade, an equal-weighted and dividend-reinvested portfolio of these five stocks would have grown from $10,000 to $89,051. That\'s a mind-blowing 24.7% compound annual growth rate. It was a bumpy ride, as the FAANG portfolio outperformed the S&P 500(SNPINDEX: ^GSPC) tracker in five years and underperformed in the other five, but stellar results of 2015, 2020, and 2023 outweighed the weak 2022 period.\n\nIn case you\'re not familiar with FAANG, I\'m talking about these familiar 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
def fill_missing_open_prices(df):
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'Open']):
            # Find the previous available Open price
            prev_open = None
            for j in range(i - 1, -1, -1):
                if pd.notna(df.loc[j, 'Open']):
                    prev_open = df.loc[j, 'Open']
                    break

            # Find the next available Open price
            next_open = None
            for j in range(i + 1, len(df)):
                if pd.notna(df.loc[j, 'Open']):
                    next_open = df.loc[j, 'Open']
                    break

            # If both previous and next are found, fill the current NaN with their mean
            if prev_open is not None and next_open is not None:
                df.loc[i, 'Open'] = (prev_open + next_open) / 2
            # If only previous is found, fill with previous value (this handles edge cases)
            elif prev_open is not None:
                df.loc[i, 'Open'] = prev_open
            # If only next is found, fill with next value (another edge case)
            elif next_open is not None:
                df.loc[i, 'Open'] = next_open

    return df

# Apply the function to fill all missing Open prices
new_df = fill_missing_open_prices(merged_df)

In [ ]:
new_df.drop(['Trend'], axis=1, inplace=True)

In [ ]:
def Add_Trend_Column(df):
    # Shift Close prices to compare with previous day's Close
    df['Prev_Open'] = df['Open'].shift(1)
    
    # Compute difference between today's and previous day's Close price
    df['Diff'] = df['Open'] - df['Prev_Open']
    
    # Determine Trend based on difference
    df['Trend'] = df['Diff'].apply(lambda x: 'rise' if x > 0 else 'fall' if x < 0 else 'equal')
    
    # Drop intermediate columns
    df = df.drop(columns=['Prev_Open', 'Diff'])
    
    return df

new_df = Add_Trend_Column(new_df)


In [ ]:
new_df

,date,ticker,news,Open,average_sentiment,Trend
0,2024-01-01,AAPL,"[{'index': 0, 'time': '06:21:00', 'title': '2 ...",187.149994,"{'polarity': 0.992625, 'neg': 0.043375, 'neu':...",equal
1,2024-01-02,AAPL,"[{'index': 8, 'time': '00:00:00', 'title': 'AA...",187.149994,"{'polarity': 0.6988644067796609, 'neg': 0.0305...",equal
2,2024-01-03,AAPL,"[{'index': 67, 'time': '00:00:00', 'title': ' ...",184.220001,"{'polarity': 0.7423684210526316, 'neg': 0.0278...",fall
3,2024-01-04,AAPL,"[{'index': 86, 'time': '00:00:00', 'title': ' ...",182.149994,"{'polarity': 0.6239166666666667, 'neg': 0.0482...",fall
4,2024-01-05,AAPL,"[{'index': 98, 'time': '00:00:00', 'title': '3...",181.990005,"{'polarity': 0.6653125, 'neg': 0.029, 'neu': 0...",fall
...,...,...,...,...,...,...
246,2024-09-03,AAPL,"[{'index': 3401, 'time': '00:00:00', 'title': ...",228.550003,"{'polarity': 0.45575000000000004, 'neg': 0.079...",fall
247,2024-09-04,AAPL,"[{'index': 3409, 'time': '00:00:00', 'title': ...",221.660004,"{'polarity': 0.573857142857143, 'neg': 0.02828...",fall
248,2024-09-05,AAPL,"[{'index': 3416, 'time': '00:00:00', 'title': ...",221.630005,"{'polarity': 0.9093333333333333, 'neg': 0.0352...",fall
249,2024-09-06,AAPL,"[{'index': 3437, 'time': '00:00:00', 'title': ...",223.949997,"{'polarity': 0.9117142857142857, 'neg': 0.0227...",rise


In [ ]:
new_df['prev_open'] = new_df['Open'].shift(1) 

In [ ]:
new_df

,date,ticker,news,Open,average_sentiment,Trend,prev_open
0,2024-01-01,AAPL,"[{'index': 0, 'time': '06:21:00', 'title': '2 ...",187.149994,"{'polarity': 0.992625, 'neg': 0.043375, 'neu':...",equal,NaN
1,2024-01-02,AAPL,"[{'index': 8, 'time': '00:00:00', 'title': 'AA...",187.149994,"{'polarity': 0.6988644067796609, 'neg': 0.0305...",equal,187.149994
2,2024-01-03,AAPL,"[{'index': 67, 'time': '00:00:00', 'title': ' ...",184.220001,"{'polarity': 0.7423684210526316, 'neg': 0.0278...",fall,187.149994
3,2024-01-04,AAPL,"[{'index': 86, 'time': '00:00:00', 'title': ' ...",182.149994,"{'polarity': 0.6239166666666667, 'neg': 0.0482...",fall,184.220001
4,2024-01-05,AAPL,"[{'index': 98, 'time': '00:00:00', 'title': '3...",181.990005,"{'polarity': 0.6653125, 'neg': 0.029, 'neu': 0...",fall,182.149994
...,...,...,...,...,...,...,...
246,2024-09-03,AAPL,"[{'index': 3401, 'time': '00:00:00', 'title': ...",228.550003,"{'polarity': 0.45575000000000004, 'neg': 0.079...",fall,228.755003
247,2024-09-04,AAPL,"[{'index': 3409, 'time': '00:00:00', 'title': ...",221.660004,"{'polarity': 0.573857142857143, 'neg': 0.02828...",fall,228.550003
248,2024-09-05,AAPL,"[{'index': 3416, 'time': '00:00:00', 'title': ...",221.630005,"{'polarity': 0.9093333333333333, 'neg': 0.0352...",fall,221.660004
249,2024-09-06,AAPL,"[{'index': 3437, 'time': '00:00:00', 'title': ...",223.949997,"{'polarity': 0.9117142857142857, 'neg': 0.0227...",rise,221.630005


In [ ]:
#drop 1st row
new_df = new_df.dropna()

In [ ]:
# Expand the average_sentiment column into separate sentiment columns
new_df[['pos', 'neg', 'neu', 'polarity']] = new_df['average_sentiment'].apply(pd.Series)

C:\Users\maazk\AppData\Local\Temp\ipykernel_44592\1019174229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['pos', 'neg', 'neu', 'polarity']] = new_df['average_sentiment'].apply(pd.Series)
C:\Users\maazk\AppData\Local\Temp\ipykernel_44592\1019174229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['pos', 'neg', 'neu', 'polarity']] = new_df['average_sentiment'].apply(pd.Series)
C:\Users\maazk\AppData\Local\Temp\ipykernel_44592\1019174229.py:2: SettingWithCopyWarning: 
A value is tryin

In [ ]:
new_df.drop(['average_sentiment'], axis=1, inplace=True)
new_df

C:\Users\maazk\AppData\Local\Temp\ipykernel_44592\801389529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(['average_sentiment'], axis=1, inplace=True)


,date,ticker,news,Open,Trend,prev_open,pos,neg,neu,polarity
1,2024-01-02,AAPL,"[{'index': 8, 'time': '00:00:00', 'title': 'AA...",187.149994,equal,187.149994,0.698864,0.030525,0.855305,0.114288
2,2024-01-03,AAPL,"[{'index': 67, 'time': '00:00:00', 'title': ' ...",184.220001,fall,187.149994,0.742368,0.027842,0.871526,0.100632
3,2024-01-04,AAPL,"[{'index': 86, 'time': '00:00:00', 'title': ' ...",182.149994,fall,184.220001,0.623917,0.048250,0.794500,0.157333
4,2024-01-05,AAPL,"[{'index': 98, 'time': '00:00:00', 'title': '3...",181.990005,fall,182.149994,0.665312,0.029000,0.886813,0.084312
5,2024-01-06,AAPL,"[{'index': 114, 'time': '00:00:00', 'title': '...",182.040001,rise,181.990005,0.753667,0.044500,0.777167,0.178500
...,...,...,...,...,...,...,...,...,...,...
246,2024-09-03,AAPL,"[{'index': 3401, 'time': '00:00:00', 'title': ...",228.550003,fall,228.755003,0.455750,0.079125,0.735000,0.185750
247,2024-09-04,AAPL,"[{'index': 3409, 'time': '00:00:00', 'title': ...",221.660004,fall,228.550003,0.573857,0.028286,0.812714,0.159143
248,2024-09-05,AAPL,"[{'index': 3416, 'time': '00:00:00', 'title': ...",221.630005,fall,221.660004,0.909333,0.035286,0.820000,0.144571
249,2024-09-06,AAPL,"[{'index': 3437, 'time': '00:00:00', 'title': ...",223.949997,rise,221.630005,0.911714,0.022714,0.854286,0.122857


In [ ]:
# Feature columns: include sentiment scores, current open price, and previous open price
feature_columns = ['Open', 'prev_open', 'pos', 'neg', 'neu', 'polarity']
# Target for price prediction (regression task)
target_price = 'Open'

# Target for trend prediction (classification task)
target_trend = 'Trend'

In [ ]:
# Split the data into X (features) and y (target)
X = new_df[feature_columns]
y_price = new_df[target_price]  # For regression
y_trend = new_df[target_trend]  # For classification

In [ ]:
X_train, X_test, y_train_price, y_test_price = train_test_split(X, y_price, test_size=0.2, random_state=42)
_, _, y_train_trend, y_test_trend = train_test_split(X, y_trend, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Initialize the KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors=3)

# Train the model
knn_regressor.fit(X_train, y_train_price)

# Make predictions on the test set
y_pred_price = knn_regressor.predict(X_test)

# Evaluate the model's performance using Mean Squared Error
mse = mean_squared_error(y_test_price, y_pred_price)
print(f"Mean Squared Error for price prediction: {mse}")


Mean Squared Error for price prediction: 0.6283317262857167


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=9)

# Train the model
knn_classifier.fit(X_train, y_train_trend)

# Make predictions on the test set
y_pred_trend = knn_classifier.predict(X_test)

# Evaluate the model's performance using accuracy score
accuracy = accuracy_score(y_test_trend, y_pred_trend)
print(f"Accuracy for trend prediction: {accuracy * 100:.2f}%")


Accuracy for trend prediction: 78.57%
